<a href="https://colab.research.google.com/github/brunorodriguesti/Experiment_CF_ML_Bug_Fixing_Time/blob/main/Experiment_ML_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd /content/gdrive/My Drive/

/content/gdrive/My Drive


In [3]:
cd Colab\ Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import nltk
import time
from datetime import timedelta

In [5]:
#Select dataset
#bugfixclass = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/eclipse_clean.csv', low_memory=False)
bugfixclass = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/netbeans_clean.csv', low_memory=False)

In [6]:
bugfixclass['classification'].value_counts()

0.0    1534
1.0    1534
Name: classification, dtype: int64

#Treating the textual features

In [13]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    #lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(stem_words)

In [15]:
bugfixclass['summary_plus_comment']= bugfixclass['short_desc']+" " +bugfixclass['comments']

In [ ]:
bugfixclass['summary_plus_comment_tok']= np.array(bugfixclass['summary_plus_comment'].map(lambda s:preprocess(s)))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
countVect = CountVectorizer(min_df=0.005)

In [ ]:
sentence_count_vect = countVect.fit_transform(bugfixclass['summary_plus_comment_tok'])

In [ ]:
 sentence_count_vect.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(sentence_count_vect)

In [ ]:
sentence = X_train_tfidf

In [ ]:
sentence.shape

In [ ]:
count_vect_df = pd.DataFrame(sentence.todense(), columns=countVect.get_feature_names())

#Features

In [7]:
features = ['reporter', 'assigned_to', 'priority_n', 'severity_n', 'product_id', 'component_id', 'month_creation', 'year_creation', 'number_cc','number_attachment' ]

In [ ]:
bugfixclass.columns

In [8]:
# Using textual information
#X = pd.concat([bugfixclass[features], count_vect_df], axis=1)

# It does not use textual information
X = bugfixclass[features]

In [9]:
y = bugfixclass['classification']

In [10]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics

#Cross validation

In [11]:
def cross_validation(X, y, c, model,v=0):
  return {'precision': cross_val_score(model, X, y, scoring="precision", cv=c, verbose=v), 'recall': cross_val_score(model, X, y, scoring="recall", cv=c, verbose = v), 'f1_score': cross_val_score(model, X, y, scoring="f1", cv=c, verbose=v)};  


In [12]:
kf = KFold(n_splits=10,shuffle=True)

#Naive Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB
#from sklearn.naive_bayes import MultinomialNB

In [14]:
gnb = GaussianNB()

# Used to classify with textual features
#classifier = MultinomialNB() 

In [15]:
start = time.time()
nbScore = cross_validation(X, y, kf, gnb)
#nbScore = cross_validation(XsD, y, kf, gnb)
end = time.time()
print('elapsed time: ', timedelta(seconds= end - start))
print ('Precision: ', round(nbScore['precision'].mean(),2)*100, ' recall: ', round(nbScore['recall'].mean(),2)*100, ' f-measure: ', round(nbScore['f1_score'].mean(),2)*100)

elapsed time:  0:00:00.146771
Precision:  70.0  recall:  21.0  f-measure:  34.0


#Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier 

In [17]:
cdt = DecisionTreeClassifier(criterion='gini', max_depth=None)

In [18]:
tart = time.time()
dtScore = cross_validation(X, y, kf, cdt)
end = time.time()
print('elapsed time: ', timedelta(seconds=end - start))
print ('Precision: ', round(dtScore['precision'].mean(),2)*100, ' recall: ', round(dtScore['recall'].mean(),2)*100, ' f-measure: ', round(dtScore['f1_score'].mean(),2)*100)

elapsed time:  0:00:05.973461
Precision:  60.0  recall:  59.0  f-measure:  59.0


#KNN

In [19]:
from sklearn.neighbors import KNeighborsClassifier

In [20]:
knn = KNeighborsClassifier()

In [21]:
start = time.time()
knnScore = cross_validation(X, y, kf, knn)
end = time.time()
print('elapsed time: ', timedelta(seconds=end - start))
print ('Precision: ', round(knnScore['precision'].mean(),2)*100, ' recall: ', round(knnScore['recall'].mean(),2)*100, ' f-measure: ', round(knnScore['f1_score'].mean(),2)*100)

elapsed time:  0:00:00.572907
Precision:  57.99999999999999  recall:  56.99999999999999  f-measure:  56.99999999999999


#Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
rfc=RandomForestClassifier(n_estimators=100)

In [25]:
start = time.time()
rfcScore = cross_validation(X, y, kf, rfc)
end = time.time()
print('elapsed time: ', timedelta(seconds=end - start))
print ('Precision: ', round(rfcScore['precision'].mean(),2)*100, ' recall: ', round(rfcScore['recall'].mean(),2)*100, ' f-measure: ', round(rfcScore['f1_score'].mean(),2)*100)

elapsed time:  0:00:13.712701
Precision:  63.0  recall:  62.0  f-measure:  63.0


In [26]:
randonForestScore = np.array(rfcScore['f1_score'])

#SVM

In [27]:
from sklearn.linear_model import SGDClassifier

In [28]:
svmclf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)

In [29]:
start = time.time()
svmScore = cross_validation(X, y, kf, svmclf)
end = time.time()
print('elapsed time: ',timedelta(seconds=end - start))
print ('Precision: ', round(svmScore['precision'].mean(),2)*100, ' recall: ', round(svmScore['recall'].mean(),2)*100, ' f-measure: ', round(svmScore['f1_score'].mean(),2)*100)

elapsed time:  0:00:00.185055
Precision:  42.0  recall:  56.99999999999999  f-measure:  48.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Collaborative Filtering

In [30]:
first = bugfixclass['days'].mode()
second = np.percentile(bugfixclass['days'], [25])
third = np.percentile(bugfixclass['days'], [50])
forth = np.percentile(bugfixclass['days'], [75])
fifth= np.percentile(bugfixclass['days'], [])
np.percentile(bugfixclass['days'], [25, 50, 75])

array([ 3. ,  6.5, 14. ])

In [31]:
bugfixclass.loc[(bugfixclass['days'] <= int(first)), 'ratings'] = 1
bugfixclass.loc[(bugfixclass['days'] > int(first)) & (bugfixclass['days'] <= int(second)) , 'ratings'] = 2
bugfixclass.loc[(bugfixclass['days'] > int(second)) & (bugfixclass['days'] <= int(third)), 'ratings'] = 3
bugfixclass.loc[(bugfixclass['days'] > int(third)) & (bugfixclass['days'] <= int(forth)), 'ratings'] = 4
bugfixclass.loc[bugfixclass['days'] > int(forth), 'ratings'] = 5

In [32]:
#bugfixclass['characteristic']= bugfixclass['bug_severity']+bugfixclass['priority']+bugfixclass['product_id'].astype(str)+bugfixclass['component_id'].astype(str)+bugfixclass['op_sys']+bugfixclass['rep_platform']+bugfixclass['number_cc'].astype(str)+bugfixclass['number_attachment'].astype(str)+bugfixclass['month_creation'].astype(str)+bugfixclass['year_creation'].astype(str)
bugfixclass['characteristic']= bugfixclass['bug_severity']+bugfixclass['priority']+bugfixclass['product_id'].astype(str)+bugfixclass['component_id'].astype(str)+bugfixclass['op_sys']+bugfixclass['rep_platform']

In [33]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [34]:
bugfixclass['characteristic_n'] = le.fit_transform(bugfixclass['characteristic'].values.reshape(-1,1))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [35]:
def precision_recall_at_k(predictions, k = 10, threshold = 3):
    
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        est = round(est, 0)
        user_est_true[uid].append((est, true_r))
        #print ('estimado '+ str(est)+ ' verdadeiro: '+ str(true_r))
    
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        # Number of relevant items
        n_rel = sum((true_r <= threshold) for (_, true_r) in user_ratings)
        
        # Number of recommended items in top k
        n_rec_k = sum((est <= threshold) for (est, _) in user_ratings[:k])
        
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r <= threshold) and (est <= threshold)) for (est, true_r) in user_ratings[:k])
        
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
        #print ("ratings: "+ str(user_ratings) + " predicted: "+ str(user_est_true) + ' true_r '+ str(true_r))
    
    return precisions, recalls;

In [36]:
def get_precision_vs_recall(algo, k_max = 10, verbose = False):
    precision_list = []
    recall_list = []
    f1_score_list = []
        
    if algo:
        for k_curr in range(1, k_max + 1):
            algo.fit(data_train)
            predictions = algo.test(data_testset)
           # print("prediction ", predictions)
            
            # Get precision and recall at k metrics for each user
            precisions, recalls = precision_recall_at_k(predictions, k = k_curr, threshold = 3)
            
            # Precision and recall can then be averaged over all users
            precision = sum(prec for prec in precisions.values()) / len(precisions)
            recall = sum(rec for rec in recalls.values()) / len(recalls)
            f1_score = 2 * (precision * recall) / (precision + recall)
            
            # Save measures
            precision_list.append(precision)
            recall_list.append(recall)
            f1_score_list.append(f1_score)

            
            
            if verbose:
                print('K =', k_curr, '- Precision:', precision, ', Recall:', recall, ', F1 score:', f1_score)
    
    return {'precision': precision_list, 'recall': recall_list, 'f1_score': f1_score_list};

In [37]:
pip install scikit-surprise

In [38]:
features = ['assigned_to','reporter', 'characteristic_n', 'ratings' ]
data = bugfixclass[features]

In [39]:
from surprise import SVD, SVDpp, SlopeOne, CoClustering, NMF, BaselineOnly, NormalPredictor
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.model_selection import GridSearchCV
#from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import timeit
import time
from collections import defaultdict
from surprise import accuracy
from surprise import dump

In [40]:
reader = Reader(rating_scale=(1, 5))
trainset, testset = train_test_split(data, test_size=0.20)

In [41]:
reader = Reader(rating_scale = (1, 5))
data_train = Dataset.load_from_df(trainset[['reporter', 'characteristic_n', 'ratings']], reader)
data_test = Dataset.load_from_df(testset[['reporter', 'characteristic_n', 'ratings']], reader)

In [42]:
data_train = data_train.build_full_trainset()
data_test = data_test.build_full_trainset()

# Create the trainset and testset
data_trainset = data_train.build_testset()
data_testset = data_test.build_testset()

In [43]:
k_max = 10

#SVD

In [44]:
algo = SVD()
start = time.time()
metrics_SVD = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_SVD['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_SVD['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_SVD['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.6551724137931034 , Recall: 0.684985632183908 , F1 score: 0.6697474098651559
K = 2 - Precision: 0.6163793103448276 , Recall: 0.7442528735632183 , F1 score: 0.6743072497542147
K = 3 - Precision: 0.6091954022988506 , Recall: 0.813731527093596 , F1 score: 0.6967631222254002
K = 4 - Precision: 0.6084770114942529 , Recall: 0.8428263546798029 , F1 score: 0.7067308923235233
K = 5 - Precision: 0.5936063218390806 , Recall: 0.8325294198139027 , F1 score: 0.6930542616451797
K = 6 - Precision: 0.5691810344827587 , Recall: 0.8406027640941434 , F1 score: 0.6787638662596984
K = 7 - Precision: 0.5857450738916257 , Recall: 0.8596401204159826 , F1 score: 0.6967415576641143
K = 8 - Precision: 0.5936268472906405 , Recall: 0.85308908045977 , F1 score: 0.7000912502274912
K = 9 - Precision: 0.5934694854953477 , Recall: 0.8591825738916256 , F1 score: 0.702024461782282
K = 10 - Precision: 0.569391420361248 , Recall: 0.8624623700054734 , F1 score: 0.6859341046822545
precision  60.0 recall  8

In [45]:
#To statistic method
svdPrediction = np.array(metrics_SVD['f1_score'])

#KNNBasic

In [46]:
algo = KNNBasic()
start = time.time()
metrics_KNN = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_KNN['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_KNN['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_KNN['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Computing the msd similarity matrix...
Done computing similarity matrix.
K = 1 - Precision: 0.6767241379310345 , Recall: 0.6875 , F1 score: 0.6820695102685623
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 2 - Precision: 0.5991379310344828 , Recall: 0.740301724137931 , F1 score: 0.6622811884484147
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 3 - Precision: 0.5833333333333333 , Recall: 0.7977729885057472 , F1 score: 0.6739055049848288
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 4 - Precision: 0.5754310344827587 , Recall: 0.8195812807881775 , F1 score: 0.6761409904184964
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 5 - Precision: 0.5757902298850573 , Recall: 0.8392515051997811 , F1 score: 0.6829944377313936
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 6 - Precision: 0.5715517241379311 , Recall: 0.8456143951833608 , F1 score: 0.6820

#KNNWithMeans

In [47]:
algo = KNNWithMeans()
start = time.time()
metrics_KNNM = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_KNNM['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_KNNM['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_KNNM['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Computing the msd similarity matrix...
Done computing similarity matrix.
K = 1 - Precision: 0.7025862068965517 , Recall: 0.6569683908045978 , F1 score: 0.6790119801393887
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 2 - Precision: 0.6314655172413793 , Recall: 0.7173132183908045 , F1 score: 0.6716573304559438
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 3 - Precision: 0.6221264367816093 , Recall: 0.7747844827586207 , F1 score: 0.6901140262987562
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 4 - Precision: 0.620330459770115 , Recall: 0.7938389436234264 , F1 score: 0.6964405759304968
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 5 - Precision: 0.6137931034482759 , Recall: 0.8120723864258348 , F1 score: 0.6991464956950018
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 6 - Precision: 0.6037356321839081 , Recall: 0.8187944718117133 , F1 s

#KNNWithZScore

In [48]:
algo = KNNWithZScore()
start = time.time()
metrics_KNNZ = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_KNNZ['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_KNNZ['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_KNNZ['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Computing the msd similarity matrix...
Done computing similarity matrix.
K = 1 - Precision: 0.6982758620689655 , Recall: 0.6526580459770116 , F1 score: 0.6746967515976127
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 2 - Precision: 0.6336206896551724 , Recall: 0.7173132183908045 , F1 score: 0.6728745106308849
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 3 - Precision: 0.6242816091954022 , Recall: 0.7747844827586207 , F1 score: 0.6914379620202734
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 4 - Precision: 0.6224856321839082 , Recall: 0.7938389436234264 , F1 score: 0.6977967411064045
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 5 - Precision: 0.615948275862069 , Recall: 0.8120723864258348 , F1 score: 0.7005425054463874
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 6 - Precision: 0.6069683908045979 , Recall: 0.8192733990147785 , F1 s

#KNNBaseline

In [49]:
algo = KNNBaseline()
start = time.time()
metrics_KNNBL = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_KNNBL['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_KNNBL['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_KNNBL['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 1 - Precision: 0.6939655172413793 , Recall: 0.6731321839080461 , F1 score: 0.6833901099896715
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 2 - Precision: 0.6228448275862069 , Recall: 0.7216235632183907 , F1 score: 0.6686055349295696
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 3 - Precision: 0.6056034482758621 , Recall: 0.7787356321839082 , F1 score: 0.6813431633949503
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 4 - Precision: 0.6045258620689655 , Recall: 0.7973111658456487 , F1 score: 0.6876622749608394
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 5 - Precision: 0.5994252873563218 , Recall: 0.8144670224411604 , F1 score: 0.6905930

#SVDpp

In [50]:
algo = SVDpp()
start = time.time()
metrics_SVDpp = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_SVDpp['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_SVDpp['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_SVDpp['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.6681034482758621 , Recall: 0.6684626436781609 , F1 score: 0.6682829977110343
K = 2 - Precision: 0.6573275862068966 , Recall: 0.7453304597701149 , F1 score: 0.6985683694644355
K = 3 - Precision: 0.6285919540229885 , Recall: 0.7873563218390804 , F1 score: 0.6990733451133389
K = 4 - Precision: 0.6235632183908046 , Recall: 0.8172037493158183 , F1 score: 0.7073707427031561
K = 5 - Precision: 0.6129310344827585 , Recall: 0.8150314723590586 , F1 score: 0.699679551942721
K = 6 - Precision: 0.6067528735632185 , Recall: 0.8221298576902025 , F1 score: 0.6982093669199503
K = 7 - Precision: 0.5892549261083745 , Recall: 0.8485050629447182 , F1 score: 0.6955066102477393
K = 8 - Precision: 0.6180162151067324 , Recall: 0.8455801860974276 , F1 score: 0.7141070663349938
K = 9 - Precision: 0.6050680760810072 , Recall: 0.8456058429118773 , F1 score: 0.705395049562558
K = 10 - Precision: 0.5951440202517789 , Recall: 0.8359845374931583 , F1 score: 0.6952990991892075
precision  62.0 recal

#SlopeOne

In [51]:
algo = SlopeOne()
start = time.time()
metrics_slope = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_slope['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_slope['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_slope['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.7025862068965517 , Recall: 0.6505028735632185 , F1 score: 0.6755421399998169
K = 2 - Precision: 0.6379310344827587 , Recall: 0.71875 , F1 score: 0.675933280381255
K = 3 - Precision: 0.6221264367816091 , Recall: 0.7833025451559933 , F1 score: 0.6934725661741036
K = 4 - Precision: 0.622485632183908 , Recall: 0.8037937876299945 , F1 score: 0.7016157943352216
K = 5 - Precision: 0.6145833333333333 , Recall: 0.8192562944718117 , F1 score: 0.7023118269949579
K = 6 - Precision: 0.6066810344827587 , Recall: 0.8282532840722496 , F1 score: 0.7003603617212278
K = 7 - Precision: 0.6090722495894909 , Recall: 0.8353687739463601 , F1 score: 0.7044938908462067
K = 8 - Precision: 0.6068041871921183 , Recall: 0.8375752599890531 , F1 score: 0.7037543712516348
K = 9 - Precision: 0.6053520114942529 , Recall: 0.8405685550082101 , F1 score: 0.7038282425207772
K = 10 - Precision: 0.6043188970990696 , Recall: 0.8440749863163656 , F1 score: 0.7043532434654886
precision  62.0 recall  80.0  f1

#CoClustering

In [52]:
algo = CoClustering()
start = time.time()
metrics_coc = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_coc['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_coc['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_coc['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.6853448275862069 , Recall: 0.6702586206896551 , F1 score: 0.6777177786305575
K = 2 - Precision: 0.6163793103448276 , Recall: 0.728448275862069 , F1 score: 0.6677442528735632
K = 3 - Precision: 0.6077586206896551 , Recall: 0.8001847290640394 , F1 score: 0.6908220665525536
K = 4 - Precision: 0.6005747126436782 , Recall: 0.8344280240831965 , F1 score: 0.6984465715078
K = 5 - Precision: 0.600359195402299 , Recall: 0.8157669677066228 , F1 score: 0.6916801809422567
K = 6 - Precision: 0.5900862068965518 , Recall: 0.829690065681445 , F1 score: 0.6896701589028967
K = 7 - Precision: 0.5927032019704433 , Recall: 0.8364463601532568 , F1 score: 0.693789438248377
K = 8 - Precision: 0.5783507799671593 , Recall: 0.8434250136836344 , F1 score: 0.686177830127766
K = 9 - Precision: 0.5808172550629448 , Recall: 0.8389094143404489 , F1 score: 0.686404043516966
K = 10 - Precision: 0.5806872605363984 , Recall: 0.8465465927750411 , F1 score: 0.68885532771579
precision  60.0 recall  80.0  

#NMF


In [53]:
algo = NMF()
start = time.time()
metrics_nmf = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_nmf['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_nmf['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_nmf['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.646551724137931 , Recall: 0.6670258620689655 , F1 score: 0.6566292325535346
K = 2 - Precision: 0.5991379310344828 , Recall: 0.7600574712643677 , F1 score: 0.6700718087045272
K = 3 - Precision: 0.5897988505747127 , Recall: 0.8036398467432949 , F1 score: 0.6803110302556232
K = 4 - Precision: 0.5768678160919539 , Recall: 0.8418685002736727 , F1 score: 0.6846188930069295
K = 5 - Precision: 0.5602011494252875 , Recall: 0.8637794198139027 , F1 score: 0.6796303745748314
K = 6 - Precision: 0.5704022988505748 , Recall: 0.864908319649699 , F1 score: 0.687441014459522
K = 7 - Precision: 0.5625718390804598 , Recall: 0.8668411330049262 , F1 score: 0.6823226316096987
K = 8 - Precision: 0.5646962233169129 , Recall: 0.8652333059660646 , F1 score: 0.6833819012214453
K = 9 - Precision: 0.5604012725779967 , Recall: 0.8812901956759716 , F1 score: 0.6851343134678591
K = 10 - Precision: 0.5666632457580733 , Recall: 0.8750684181718664 , F1 score: 0.6878798912552
precision  57.99999999999

#BaselineOnly



In [54]:
algo = BaselineOnly()
start = time.time()
metrics_bl = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_bl['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_bl['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_bl['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Estimating biases using als...
K = 1 - Precision: 0.6120689655172413 , Recall: 0.7157738095238094 , F1 score: 0.6598717007381071
Estimating biases using als...
K = 2 - Precision: 0.5732758620689655 , Recall: 0.8022372742200329 , F1 score: 0.6687006511666199
Estimating biases using als...
K = 3 - Precision: 0.5567528735632185 , Recall: 0.8544574438970991 , F1 score: 0.6742037403515078
Estimating biases using als...
K = 4 - Precision: 0.5499281609195402 , Recall: 0.8746921182266011 , F1 score: 0.6752926867438305
Estimating biases using als...
K = 5 - Precision: 0.5423132183908046 , Recall: 0.8840483032293377 , F1 score: 0.6722434295516708
Estimating biases using als...
K = 6 - Precision: 0.5361350574712643 , Recall: 0.8916256157635468 , F1 score: 0.6696244821860603
Estimating biases using als...
K = 7 - Precision: 0.539090722495895 , Recall: 0.8996091611932129 , F1 score: 0.6741794562852327
Estimating biases using als...
K = 8 - Precision: 0.5377257799671593 , Recall: 0.9032481527093597 

#NormalPredictor

In [55]:
algo = NormalPredictor()
start = time.time()
metrics_norm = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_norm['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_norm['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_norm['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.7370689655172413 , Recall: 0.5380747126436781 , F1 score: 0.6220446818844099
K = 2 - Precision: 0.7025862068965517 , Recall: 0.552801724137931 , F1 score: 0.6187583247003108
K = 3 - Precision: 0.7277298850574712 , Recall: 0.6519396551724138 , F1 score: 0.6877530546104192
K = 4 - Precision: 0.6752873563218391 , Recall: 0.6532738095238094 , F1 score: 0.6640982065840124
K = 5 - Precision: 0.6933908045977012 , Recall: 0.6998836891078271 , F1 score: 0.6966221179067263
K = 6 - Precision: 0.6334770114942527 , Recall: 0.7067084017515053 , F1 score: 0.6680919250645937
K = 7 - Precision: 0.6655890804597702 , Recall: 0.6862342638204708 , F1 score: 0.6757540244719543
K = 8 - Precision: 0.6832820197044334 , Recall: 0.7041256157635467 , F1 score: 0.6935472467718702
K = 9 - Precision: 0.7004515599343186 , Recall: 0.6693820128626163 , F1 score: 0.6845644382101873
K = 10 - Precision: 0.6255336617405584 , Recall: 0.6707204433497537 , F1 score: 0.6473394580356463
precision  68.0 reca

# Wilcoxon Test

In [56]:
from scipy.stats import wilcoxon, friedmanchisquare, rankdata

In [57]:
wilcoxon(randonForestScore, svdPrediction, zero_method='zsplit')

WilcoxonResult(statistic=0.0, pvalue=0.005062032126267864)

In [58]:
stat, p = wilcoxon(randonForestScore, svdPrediction)
print('stat=%.3f, p=%.3f' % (stat, p))
if p >= 0.05:
	print('statistically significant')
else:
	print('It is not statistically significant')

stat=0.000, p=0.005
It is not statistically significant
